##### <center>Downloading Meta's CHM data from Google Earth Engine</center>

###### See example here: https://code.earthengine.google.com/?scriptPath=users%2Fsat-io%2Fawesome-gee-catalog-examples%3Aagriculture-vegetation-forestry%2FGLOBAL-1m-CANOPY-HEIGHT

In [1]:
import ee
import geemap
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

ee.Initialize(project = 'ee-zack')

In [2]:
# store colormap
viridis = plt.get_cmap('viridis', 7)

# convert colormap to list of hex codes
palette = [mpl.colors.rgb2hex(viridis(i)) for i in range(viridis.N)]

In [3]:
# Load the counties for each state
counties_illinois = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '17'))
counties_iowa = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '19'))

# pull meta image collection and clip to counties
dataset_illinois = ee.ImageCollection('projects/meta-forest-monitoring-okw37/assets/CanopyHeight').map(lambda img: img.clip(counties_illinois))
dataset_iowa = ee.ImageCollection('projects/meta-forest-monitoring-okw37/assets/CanopyHeight').map(lambda img: img.clip(counties_iowa))

In [4]:
# create a map and add the illinois and iowa assets
Map = geemap.Map()
Map.centerObject(counties_illinois, 6)

# add illinois and iowa canopy height to map
Map.addLayer(dataset_illinois, {'min':0, 'max':25, 'palette':palette}, 'Illinois Canopy Height [meters]')
Map.addLayer(dataset_iowa, {'min':0, 'max':25, 'palette':palette}, 'Iowa Canopy Height [meters]')

# add illinois and iowa counties to map with no fill (opacity=0)
Map.addLayer(counties_illinois.style(**{'color': '000000', 'fillColor': '00000000'}), {}, 'Illinois Counties')
Map.addLayer(counties_iowa.style(**{'color': '000000', 'fillColor': '00000000'}), {}, 'Iowa Counties')

# display map
Map

Map(center=[40.10027731398491, -89.14995333112633], controls=(WidgetControl(options=['position', 'transparent_…

#### <center>Divide states into subregions and export canopy height data as tiff</center>

In [5]:
# Define the output directory
out_dir = "C:/Users/exx/Documents/GitHub/Savanna-Institute/HighResCanopyHeight/chm_data"

# store the counties feature collections as lists
counties_illinois = counties_illinois.toList(counties_illinois.size())
counties_iowa = counties_iowa.toList(counties_iowa.size())

# Define a function to split the region into smaller chunks
def split_region(region, num_splits):
    coords = region.coordinates().getInfo()[0]
    min_lon = min([coord[0] for coord in coords])
    max_lon = max([coord[0] for coord in coords])
    min_lat = min([coord[1] for coord in coords])
    max_lat = max([coord[1] for coord in coords])
    
    lon_step = (max_lon - min_lon) / num_splits
    lat_step = (max_lat - min_lat) / num_splits
    
    sub_regions = []
    for i in range(num_splits):
        for j in range(num_splits):
            sub_region = ee.Geometry.Rectangle([
                min_lon + i * lon_step,
                min_lat + j * lat_step,
                min_lon + (i + 1) * lon_step,
                min_lat + (j + 1) * lat_step
            ])
            sub_regions.append(sub_region)
    
    return sub_regions

# Define a function to check and further split regions if necessary
def check_and_split_region(region, max_dim=32768, scale=1, max_request_size=50331648):
    # Get the dimensions of the region in meters
    info = region.bounds().getInfo()
    width = info['coordinates'][0][1][0] - info['coordinates'][0][0][0]
    height = info['coordinates'][0][2][1] - info['coordinates'][0][1][1]
    
    # Convert degrees to meters (approximation)
    width_meters = width * 111320
    height_meters = height * 111320
    
    # Calculate the number of splits needed
    num_splits = 1
    while (width_meters / num_splits) / scale > max_dim or (height_meters / num_splits) / scale > max_dim:
        num_splits *= 2
    
    # Further split if the request size exceeds the limit
    while (width_meters * height_meters * 4 / (num_splits ** 2)) > max_request_size:
        num_splits *= 2
    
    # Split the region
    return split_region(region, num_splits)

# Function to export subregions for a given county
def export_subregions(image, county_name, state_name, out_dir):
    county_geometry = image.geometry()
    sub_regions = check_and_split_region(county_geometry, max_dim=32768, scale=1, max_request_size=50331648)
    
    # Create the county directory if it doesn't exist
    county_dir = os.path.join(out_dir, state_name, county_name)
    os.makedirs(county_dir, exist_ok=True)
    
    for idx, sub_region in enumerate(sub_regions):
        filename = os.path.join(county_dir, f"{county_name}_sub_region_{idx}.tif")
        try:
            geemap.ee_export_image(
                ee_object=image,
                filename=filename,
                scale=1,  # Set the scale to 1 meter
                region=sub_region,
                file_per_band=False,
                format="ZIPPED_GEO_TIFF",
                unzip=True
            )
        except Exception as e:
            print(f"Failed to export subregion {idx} for county {county_name}: {e}")

In [ ]:
# Process each county in Illinois
for i in range(counties_illinois.size().getInfo()):
    county = ee.Feature(counties_illinois.get(i))
    county_name = county.get('NAME').getInfo()
    county_geometry = county.geometry()
    county_image = dataset_illinois.mosaic().clip(county_geometry)
    export_subregions(county_image, county_name, 'Illinois', out_dir)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\exx\Documents\GitHub\Savanna-Institute\HighResCanopyHeight\chm_data\Illinois\Marion\Marion_sub_region_0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\exx\Documents\GitHub\Savanna-Institute\HighResCanopyHeight\chm_data\Illinois\Marion\Marion_sub_region_1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\exx\Documents\GitHub\Savanna-Institute\HighResCanopyHeight\chm_data\Illinois\Marion\Marion_sub_region_2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\exx\Documents\GitHub\Savanna-Institute\HighResCanopyHeight\chm_data\Illinois\Marion\Marion_sub_region_3.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\exx\Documents\GitHub\Savanna-Institute\HighResCanopyHeight\chm_data\Illinois\Marion\Marion_sub_region_4.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\exx\Documents\GitHub\Savanna-Institute\HighResCanopyHeight\chm_data\Illinois\Mario

In [ ]:
# Process each county in Iowa
for i in range(counties_iowa.size().getInfo()):
    county = ee.Feature(counties_iowa.get(i))
    county_name = county.get('NAME').getInfo()
    county_geometry = county.geometry()
    county_image = dataset_iowa.mosaic().clip(county_geometry)
    export_subregions(county_image, county_name, 'Iowa', out_dir)